In [1]:
#Importa as bibliotecas necessárias
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

In [5]:
# O dataset é do próprio Scikit Learn
data = fetch_california_housing(as_frame=True)
df = data.frame

print("Visualizar as 5 primeiras linhas do dataset:")
display(df.head())


Visualizar as 5 primeiras linhas do dataset:


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [7]:
#Análise básica do dataset
print("Informações do conjunto de dados")
df.info()

print("Estatísticas descritivas")
display(df.describe())

Informações do conjunto de dados
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB
Estatísticas descritivas


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


In [8]:
#Separação entre as variáveis independentes(X) e a alvo(y)
X = df.drop("MedHouseVal", axis=1)
y = df["MedHouseVal"]

In [9]:
# Cria novas features para melhorar o desempenho do algoritmo
X["Rooms_per_Household"] = df["AveRooms"] / df["HouseAge"]
X["Bedrooms_per_Room"] = df["AveBedrms"] / df["AveRooms"]
X["Population_per_Household"] = df["Population"] / df["HouseAge"]

In [10]:
# Divisão dos conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [11]:
#Escalonamento e padronização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
resultados = {}

#Utilizando a Regressão Linear
modelo_lr = LinearRegression()
modelo_lr.fit(X_train_scaled, y_train)
y_pred_lr = modelo_lr.predict(X_test_scaled)

resultados["Regressão Linear"] = {
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred_lr)),
    "R2": r2_score(y_test, y_pred_lr)
}

#Utilizando o XGBoost
modelo_xgb = XGBRegressor(n_estimators=200, learning_rate=0.1, random_state=42)
modelo_xgb.fit(X_train, y_train)
y_pred_xgb = modelo_xgb.predict(X_test)

resultados["XGBoost"] = {
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred_xgb)),
    "R2": r2_score(y_test, y_pred_xgb)
}

#Utilizando Rede Neural Artificial (MLP)
modelo_ann = MLPRegressor(hidden_layer_sizes=(100, 50),
                          activation='relu',
                          solver='adam',
                          max_iter=500,
                          random_state=42)
modelo_ann.fit(X_train_scaled, y_train)
y_pred_ann = modelo_ann.predict(X_test_scaled)

resultados["Rede Neural (ANN)"] = {
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred_ann)),
    "R2": r2_score(y_test, y_pred_ann)
}


In [13]:
#Avaliação dos resultados
print("Resultados")
resultados_df = pd.DataFrame(resultados).T
display(resultados_df.sort_values(by="RMSE"))

Resultados


,RMSE,R2
XGBoost,0.464398,0.835421
Rede Neural (ANN),0.518464,0.794870
Regressão Linear,0.729232,0.594188


In [15]:
melhor_modelo = resultados_df["RMSE"].idxmin()
print(f"Resposta: O modelo com menor erro (RMSE) foi o {melhor_modelo}")

O modelo com menor erro (RMSE) foi o XGBoost


In [16]:
"""
Resposta 2:
Para otimizar mais o desempenho podemos
1. Aplicar otimização de hiperparâmetros (GridSearchCV ou RandomizedSearchCV).
2. Aumentar a complexidade do modelo (mais camadas na ANN, mais árvores no XGBoost).
3. Adicionar novas variáveis derivadas (feature engineering).
4. Reduzir outliers ou aplicar transformações logarítmicas.
5. Usar técnicas de validação cruzada (cross-validation).
"""

'\nResposta 2:\nPara otimizar mais o desempenho podemos\n1. Aplicar otimização de hiperparâmetros (GridSearchCV ou RandomizedSearchCV).\n2. Aumentar a complexidade do modelo (mais camadas na ANN, mais árvores no XGBoost).\n3. Adicionar novas variáveis derivadas (feature engineering).\n4. Reduzir outliers ou aplicar transformações logarítmicas.\n5. Usar técnicas de validação cruzada (cross-validation).\n'